In [1]:
import numpy as np, pandas as pd
from tqdm import tqdm_notebook
import json

from pathlib import Path
import os
import csv
import operator

## Prepare the data

In [2]:
path = '../resources/data/training/uzbek.csv'
data = pd.read_csv(path)

In [3]:
data.head()

,label,text
0,History<SEP>Education,"Wild Field Wild Field. Tarixiy tasodifiylik, X..."
1,Offers & Promotions,"PASASHOESS Hurmatli mijozlar, PASHOES sizni o'..."
2,Travel & Tourism<SEP>Offers & Promotions,Rossiyaning arzon Piratesru qaroqchilari - arz...
3,Offers & Promotions<SEP>Travel & Tourism,TripToDream sayohat kanali haqida e'lon! Ertag...
4,Offers & Promotions<SEP>Travel & Tourism,Vandrouki Travel kanali haqida e'lon! Ertaga S...


In [4]:
mapper = {
    "Art & Design":"__label__0",
    "Bets & Gambling":"__label__1",
    "Books":"__label__2",
    "Business & Entrepreneurship":"__label__3",
    "Cars & Other Vehicles":"__label__4",
    "Celebrities & Lifestyle":"__label__5",
    "Cryptocurrencies":"__label__6",
    "Culture & Events":"__label__7",
    "Curious Facts":"__label__8",
    "Directories of Channels & Bots":"__label__9",
    "Drug Sale":"__label__10",
    "Economy & Finance":"__label__11",
    "Education":"__label__12",
    "Erotic Content":"__label__13",
    "Fashion & Beauty":"__label__14",
    "Fitness":"__label__15",
    "Forgery":"__label__16",
    "Food & Cooking":"__label__17",
    "Foreign Language Learning":"__label__18",
    "Hacked Accounts & Software":"__label__19",
    "Health & Medicine":"__label__20",
    "History":"__label__21",
    "Hobbies & Activities":"__label__22",
    "Home & Architecture":"__label__23",
    "Humor & Memes":"__label__24",
    "Investments":"__label__25",
    "Job Listings":"__label__26",
    "Kids & Parenting":"__label__27",
    "Marketing & PR":"__label__28",
    "Motivation & Self-development":"__label__29",
    "Movies":"__label__30",
    "Music":"__label__31",
    "Offers & Promotions":"__label__32",
    "Personal Data":"__label__33",
    "Pets":"__label__34",
    "Pirated Content":"__label__35",
    "Politics & Incidents":"__label__36",
    "Prostitution":"__label__37",
    "Psychology & Relationships":"__label__38",
    "Real Estate":"__label__39",
    "Recreation & Entertainment":"__label__40",
    "Religion & Spirituality":"__label__41",
    "Science":"__label__42",
    "Spam & Fake Followers":"__label__43",
    "Sports":"__label__44",
    "Technology & Internet":"__label__45",
    "Travel & Tourism":"__label__46",
    "Video Games":"__label__47",
    "Weapon Sale":"__label__48",
    "Other":"__label__49"
    }

reverse_mapper = {v: k for k, v in mapper.items()}

In [5]:
def replace_words(s, words):
    for k, v in words.items():
        s = s.replace(k, v)
    return s

In [6]:
y_data_preprocesse = data.label.apply(lambda x: x.split("<SEP>"))
y_data = []
for i in y_data_preprocesse:
    y_data.append([replace_words(item, mapper) for item in i])

print(y_data_preprocesse[0])
y_data[0]

['History', 'Education']


['__label__21', '__label__12']

In [7]:
data.label = pd.Series(y_data).apply(lambda x: ' '.join(x))
data.label

0                   __label__21 __label__12
1                               __label__32
2                   __label__46 __label__32
3                   __label__32 __label__46
4                   __label__32 __label__46
                        ...                
31496                           __label__25
31497                            __label__6
31498                           __label__25
31499    __label__25 __label__6 __label__45
31500                __label__0 __label__40
Name: label, Length: 31501, dtype: object

In [8]:
data

,label,text
0,__label__21 __label__12,"Wild Field Wild Field. Tarixiy tasodifiylik, X..."
1,__label__32,"PASASHOESS Hurmatli mijozlar, PASHOES sizni o'..."
2,__label__46 __label__32,Rossiyaning arzon Piratesru qaroqchilari - arz...
3,__label__32 __label__46,TripToDream sayohat kanali haqida e'lon! Ertag...
4,__label__32 __label__46,Vandrouki Travel kanali haqida e'lon! Ertaga S...
...,...,...
31496,__label__25,SUNSHINE INVESTIVE GROUP ðð [SI G] Hisob...
31497,__label__6,RektChat kundalik o'quv xaritalarini sizning s...
31498,__label__25,â FOREX IQ OPTION TRADINGâ qonuniy investit...
31499,__label__25 __label__6 __label__45,P2PB2B p2pb2b - bu dunyodagi eng tez rivojlana...


In [9]:
data.label.iloc[0]

'__label__21 __label__12'

In [10]:
# Remove emojis
import re


def deEmojify(text):    
    regex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"                    
                      "]+", re.UNICODE)
    return regex_pattern.sub(r'',text)

data['text'] = data['text'].apply(deEmojify)

In [11]:
data['text'] = data['text'].apply(lambda x: x.lower())

In [12]:
def removeEmail(text):
    pattern = re.compile("((\w+)(\.|_)?(\w*)@(\w+)(\.(\w+))+)")
    return pattern.sub(r'', text)


data['text'] = data['text'].apply(removeEmail)

In [13]:
def removeUsername(text):
    pattern = re.compile("(@(\w+))")
    return pattern.sub(r'', text)


data['text'] = data['text'].apply(removeUsername)

In [14]:
def removeLinks(text):
    pattern = re.compile("(https?://[^ ]+)")
    return pattern.sub(r'', text)


data['text'] = data['text'].apply(removeLinks)

In [15]:
data.to_csv('../resources/data/category_uz/fasttext_train.txt', header=None, index=None, sep=' ', mode='a')

## Train fasttext

In [16]:
import fasttext

In [17]:
fasttext_params = {
    'input': "../resources/data/category_uz/fasttext_train.txt",
    'lr': 0.1,
    'lrUpdateRate': 1000,
    'thread': 8,
    'epoch': 10,
    'wordNgrams': 1,
    'dim': 100,
    'loss': 'ova'
}

In [18]:
model = fasttext.train_supervised(**fasttext_params)

In [19]:
model.test("../resources/data/category_uz/fasttext_train.txt")

(31501, 0.4248119107329926, 0.2878840030978401)

## Evaluate model

In [20]:
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import accuracy_score, roc_auc_score
y_pred = model.predict("путешествуй дешево piratesru пираты россия - дешевые авиабилеты и самостоятельные путешествия!  обсуждение: vk.com/piratesru  по вопросам сотрудничества:  анонс! завтра распродажа smartavia: миллион билетов по россии от 990₽#разное новый маршрут nordwind: из минвод в стамбул от 5600₽ туда-обратно, можно на 23 февраля и 8 марта#билеты #юфо pegasus объявил о скидке 25%: хм... в стамбул из минвод и грозного от 8500₽/9900₽ туда-обратно#low_cost #pegasus #апрель #без_багажа #июнь #лоукост #май #стамбул #турция новое направление азимута! из воронежа в ростов за 1800₽ туда-обратно с конца марта#москва #билеты #юфо туры на неделю в турцию из мск или спб от 14200₽/15500₽ на чел. в конце февраля-начале марта#москва #спб #туры цены вниз! с nordwind из москвы в горно-алтайск от 8300₽ туда-обратно#москва #билеты #сибирь подешевело! прямые рейсы nordwind из мск в хабаровск и владивосток от 15300₽ туда-обратно#москва #билеты #дальний_восток летим в египет на новый год! дешевые билеты в хургаду и шарм-эль-шейх от 13400₽ туда-обратно#москва #билеты неожиданно! прямые рейсы из ростова в каир от 20400₽ туда-обратно с багажом#билеты #юфо актуально! в мае и начале июня из спб в измир за 12700₽ туда-обратно. прямые рейсы#спб #билеты ", k=2)
print(y_pred)
#print('Accuracy ', accuracy_score(y_test, y_pred))
#print('F1 ', f1_score(y_test, y_pred.astype(float), average='micro'))
#print('Precision ', precision_score(y_test, y_pred.astype(float), average='micro'))
#print('Recall ', recall_score(y_test, y_pred.astype(float), average='micro'))
#print(y_pred[0])
#print(y_test.iloc[0].values)

(('__label__49"', '__label__49'), array([0.11921292, 0.09010299]))


## Save lightmgbm models

In [21]:
model.save_model('../resources/models/uz/fasttext_50_uz.ftz')